In [ ]:
import boto3
import subprocess
import json
from shapely.geometry import box, mapping
import geopandas as gpd
import os
from concurrent.futures import ThreadPoolExecutor
from GSV import geometry

In [ ]:
# trajectory_file='/home/ubuntu/lavender_floor_height/output/panorama/FramePosOptimised-wagga-wagga_EPSG7844.geojson' # trajectory file for panorama images
# building_points_file='/home/ubuntu/lavender_floor_height/input/Final_Wagga_training_samples.geojson' # copy training building points samples from local computer if needed through VS Code remote explorer
# out_epsg='EPSG:28355'
# out_file='/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata.geojson' # ouput buildings points with panorama metadata joined

In [ ]:
trajectory_file='/home/ubuntu/lavender_floor_height/output/panorama/FramePosOptimised-wagga-wagga_EPSG7844.geojson' # trajectory file for panorama images
building_points_file='/mnt/floorheightvolume/all_buildings/Final_Wagga.geojson' # copy training building points samples from local computer if needed through VS Code remote explorer
out_epsg='EPSG:28355'
out_file='/mnt/floorheightvolume/all_buildings/Final_Wagga_pano_metadata.geojson' # ouput buildings points with panorama metadata joined

In [ ]:
gdf_buildings=gpd.read_file(building_points_file).to_crs(out_epsg)
gdf_buildings.head()

In [ ]:
gdf_buildings=gdf_buildings[gdf_buildings["USAGE"]=="Residential"].reset_index(drop=True)

In [ ]:
len(gdf_buildings)

In [ ]:
gdf_panorama = gpd.read_file(trajectory_file).to_crs(out_epsg)
gdf_panorama.head()

In [ ]:
len(gdf_panorama)

In [ ]:
# identify nearest panorama images
# Perform the spatial join with a 40-meter distance tolerance
nearby_points = gpd.sjoin_nearest(gdf_buildings, gdf_panorama, max_distance=60, how='inner', distance_col="distance")
nearby_points

In [ ]:
len(nearby_points[nearby_points['distance']>40]['UFI'])

### Save file with panorama frame ID

In [ ]:
nearby_points.to_file(out_file)

In [ ]:
nearby_points.columns

In [ ]:
panorama_names=nearby_points['IMGID'].to_list()
panorama_names

In [ ]:
mfa_token = '904846' # update every time you run
cmd = (f'aws sts get-session-token --serial-number arn:aws:iam::693903849513:mfa/Lavender_AWS_MFA  --token-code {mfa_token}').split()
result = subprocess.run(cmd, capture_output=True)
# Check if the command was successful
if result.returncode == 0:
    # Parse JSON output into a dictionary
    data = json.loads(result.stdout)
    #print(data)  # Print or process the dictionary
else:
    print("Error:", result.stderr)

session = boto3.session.Session(
    aws_access_key_id=data['Credentials']['AccessKeyId'],
    aws_secret_access_key=data['Credentials']['SecretAccessKey'],
    aws_session_token=data['Credentials']['SessionToken']
)
# Initialize an S3 client
s3_client = session.client('s3')

In [ ]:
s3_bucket = 'frontiersi-p127-floor-height-woolpert'
s3_prefix='01_WaggaWagga/01_StreetViewImagery/'
download_dir='/mnt/floorheightvolume/panorama_Wagga'
os.makedirs(download_dir, exist_ok=True)

In [ ]:
panorama_names

In [ ]:
# panorama_names_set=set(panorama_names)  # Convert file_names to a set for faster lookup
# extract frame ids in case not exact match
frame_ids=[panorama_name.split('_')[1] for panorama_name in panorama_names]
frame_ids_set=set(frame_ids)

In [ ]:
paginator = s3_client.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=s3_bucket, Prefix=s3_prefix)

In [ ]:
# Function to download a single file
def download_file(obj_key):
    file_name = os.path.basename(obj_key)
    local_path = os.path.join(download_dir, file_name)
    # Check if file already exists
    if not os.path.exists(local_path):
        s3_client.download_file(s3_bucket, obj_key, local_path)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Skipping (already exists): {file_name}")

In [ ]:
# Use ThreadPoolExecutor for concurrent downloads
with ThreadPoolExecutor(max_workers=10) as executor:
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                obj_key = obj['Key']
                file_name = os.path.basename(obj_key)
                if file_name.endswith('.jpg'):
                    frame_id=file_name.split('_')[1]
                    # if file_name in panorama_names_set:# exact file name matching
                    if frame_id in frame_ids_set:
                        # Submit download task to the thread pool
                        executor.submit(download_file, obj_key)
                        # panorama_names_set.remove(file_name)  # Remove to avoid redundant searches
                        frame_ids_set.remove(frame_id)

# # Check if any files were not found
# if panorama_names_set:
#     print(f"Files not found: {panorama_names_set}")

# Check if any files were not found
if frame_ids_set:
    print(f"Files not found: {frame_ids_set}")